# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [10]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url	slides_url
2009-10-01	Paper Title Number 1	Journal 1	This paper is about the number 1. The number 2 is left for future work.	Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).	paper-title-number-1	http://academicpages.github.io/files/paper1.pdf	http://academicpages.github.io/files/slides1.pdf
2010-10-01	Paper Title Number 2	Journal 1	This paper is about the number 2. The number 3 is left for future work.	Your Name, You. (2010). "Paper Title Number 2." <i>Journal 1</i>. 1(2).	paper-title-number-2	http://academicpages.github.io/files/paper2.pdf	http://academicpages.github.io/files/slides2.pdf
2015-10-01	Paper Title Number 3	Journal 1	This paper is about the number 3. The number 4 is left for future work.	Your Name, You. (2015). "Paper Title Number 3." <i>Journal 1</i>. 1(3).	paper-title-number-3	http://academicpages.github.io/files/paper3.pdf	http://academicpages.github.io/files/slides3.pdf

## Import pandas

We are using the very handy pandas library for dataframes.

In [1]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("my_pub.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2024-01-01,A Causal Regulation Modeling Algorithm for Tem...,International Journal of Molecular Sciences,NaN,"Chen, Y., Mao, R., Xu, J., Huang, Y., Xu, J., ...",chen2024-causal,NaN,NaN
1,2022-01-01,CircNet 2.0: An Updated Database for Exploring...,Nucleic Acids Research,NaN,"Chen, Y., Yao, L., Tang, Y., Jhong, J.-H., Wan...",chen2022-circnet,NaN,NaN
2,2023-01-01,Quantitative Model for Genome-Wide Cyclic AMP ...,Briefings in Bioinformatics,NaN,"Chen, Y., Lin, Y.-C.-D., Luo, Y., Cai, X., Qiu...",chen2023-crp,NaN,NaN
3,2024-11-01,DeepDrugmiR: A Two-Stage Deep Learning Method ...,Proceedings of the 15th ACM International Conf...,NaN,"Huang, Y., Wu, H., Cai, Y., Dong, D., Yu, S., ...",huang2024-deepdrugmir,NaN,NaN
4,2021-01-01,MethHC 2.0: Information Repository of DNA Meth...,Nucleic Acids Research,NaN,"Huang, H.-Y., Li, J., Tang, Y., Huang, Y.-X., ...",huang2021-methhc,NaN,NaN
5,2020-01-01,miRTarBase 2020: Updates to the Experimentally...,Nucleic Acids Research,NaN,"Huang, H.-Y., Lin, Y.-C.-D., Li, J., Huang, K....",huang2020-mirtarbase,NaN,NaN
6,2022-01-02,miRTarBase Update 2022: An Informative Resourc...,Nucleic Acids Research,NaN,"Huang, H.-Y., Lin, Y.-C.-D., Cui, S., Huang, Y...",huang2022-mirtarbase,NaN,NaN
7,2023-01-02,A Robust Drug--Target Interaction Prediction F...,International Journal of Molecular Sciences,NaN,"Huang, Y., Huang, H.-Y., Chen, Y., Lin, Y.-C.-...",huang2023-capsule,NaN,NaN
8,2023-06-01,Panax notoginseng Alleviates Oxidative Stress ...,Chinese Medicine,NaN,"Tang, Y., Chen, Y.-G., Huang, H.-Y., Li, S.-F....",tang2023-panax,NaN,NaN
9,2024-01-02,miRStart 2.0: Enhancing miRNA Regulatory Insig...,Nucleic Acids Research,NaN,"Xu, J., Wan, J., Huang, H.-Y., Chen, Y., Huang...",xu2024-mirstart,NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [15]:
!ls ../_publications/

2009-10-01-paper-title-number-1.md  2015-10-01-paper-title-number-3.md
2010-10-01-paper-title-number-2.md  2024-02-17-paper-title-number-4.md


In [16]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
slidesurl: 'http://academicpages.github.io/files/slides1.pdf'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---
This paper is about the number 1. The number 2 is left for future work.

[Download slides here](http://academicpages.github.io/files/slides1.pdf)

[Download paper here](http://academicpages.github.io/files/paper1.pdf)

Recommended citation: Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).